In [8]:
import paho.mqtt.client as mqtt
import paho.mqtt.subscribe as subscribe
import time
from datetime import datetime
import json
import pymongo
from bson.objectid import ObjectId
from functions import addDocumentToDatabase, convertPayloadToCorrectFormat

#myclient = pymongo.MongoClient("mongodb://localhost:27017/")


#en funktion som adderar valt information till databasen
# def addDocumentToDatabase(databaseToAddTo:str, collectionToAddTo:str , documentToAddToDatabase:dict):
#     database = myclient[databaseToAddTo]
#     collection = database[collectionToAddTo]
#     collection.insert_one(documentToAddToDatabase)

In [9]:
# myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# mydb = myclient["testData"]
# mycol = mydb["sensordata"]

m = subscribe.simple(topics=['#'], hostname="eu1.cloud.thethings.network", port=1883, auth={'username':"ap-addiva-01@ttn",'password':"NNSXS.GB7LQUCAHNXYDZ65KIS4FV2JDQYVJMYK42DBT2Q.CJ3VVVVKOPIYACEXK45WK6XPTL2QIGC6DIL6JZ2YK3OHLMAEEC7Q"}, msg_count=2)

In [13]:
# now = datetime.now()
def convertPayloadToCorrectFormat1(inData) -> dict:
    document = {}
    now = datetime.now()
    dateAndTime = now.strftime("%Y/%m/%d, %H:%M:%S")
    sensorData = inData[2:-1]
    sensorDataToDict = json.loads(sensorData)
    payloadLvl1 = sensorDataToDict["uplink_message"]
    payloadLvl2 = payloadLvl1["decoded_payload"]
    idLvl1 = sensorDataToDict["end_device_ids"]
    idLvl2 = idLvl1["device_id"]
    document["sensor_id"] = str(idLvl2)
    document.update(payloadLvl2)
    document["time"] = str(dateAndTime)

    if idLvl2 == "eui-a81758fffe075b66":
        fixed_document = {"Sensor address": "Skrivbord Addiva Sigurdsgatan"}
        fixed_document.update(document)
    elif idLvl2 == "eui-a81758fffe075b65":
        fixed_document = {"Sensor address": "Pingisrum Addiva Sigurdsgatan"}
        fixed_document.update(document)
    elif idLvl2 == "eui-a81758fffe075c91":
        fixed_document = {"Sensor address": "Matsal Addiva Sigurdsgatan"}
        fixed_document.update(document)
    return fixed_document


for a in m:
    fixed_dictionary = convertPayloadToCorrectFormat1(str(a.payload))
    addDocumentToDatabase("test_vecka_2_Data", "sensordata", fixed_dictionary)
    # print("---------------------------TOPIC-START----------------------------")
    # print(a.topic)
    # print("---------------------------TOPIC-END----------------------------")
    # print("---------------------------PAYLOAD-START----------------------------")
    # print(a.payload)
# def addDocumentToDatabase(databaseToAddTo:str, collectionToAddTo:str , documentToAddToDatabase:dict):

    # document = str(a.payload)
    # document = document[717:776] 
    # document = "{" + document + "}"
    # print(document)
    # dateAndTime = now.strftime("%Y/%m/%d, %H:%M:%S")
    # print(dateAndTime)
    
    # document = json.loads(document)
    # document["time"] = str(dateAndTime)

    # fixed_dictionary = {"Sensor address": "Addiva Test Sigurdsgatan"}
    # fixed_dictionary.update(document)
    

    # addDocumentToDatabase("testData", "sensordata", fixed_dictionary)
    # print("---------------------------PAYLOAD-END----------------------------")
    # #client.loop_stop() #stop the loop
    # print(fixed_dictionary)

In [16]:
from functions import removeDocumentById
removeDocumentById("testData", "sensordata" ,"6344695eb42d5f4d999a9d4b", True)